### Word Group In Youtube

In [109]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [110]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [111]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 40000  # 200  # native word end index
unique_word_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000 = 90 ; 40000 = 90
include_wordgroup_ratio = 90  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000 = 90 ; 40000 = 90
word_usage_min = 5  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
word_usage_max = 100

# sentence ratio and time shift
adjust_sent_word_ratio = 30
adjust_text_word_ratio = 45  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

6


In [112]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [113]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [114]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [115]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [116]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [117]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [118]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987921,karneleme,5
987922,karnaya,5
987923,dörtlümüzün,5
987924,karnavalınız,5


In [119]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [120]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
19995,dönerler,2469
19996,katilsin,2469
19997,hangisine,2469
19998,taa,2469


In [121]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [122]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [123]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

19990

#### Youtube Sentence Analysis

In [124]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [125]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [126]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,3,10.267,11.394,mumu,fI1BI4d5KFU
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [127]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [128]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,57.2,"lm, betimlemesi, staff","bu, tarafından, dizinin, fi"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,25.1,"yaptırılmıştır, betimleme, derneğine",sesli
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.1,wwwsebederorg,
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.1,mumu,
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,71.5,"safhada, katsayım","hat, bak, şu, an, sinir"
...,...,...,...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,62.6,"bitirmeden, engebeli, yolların, arazilerin, bu...","isterim, ki, videoyu, olduğu, da, bir, dümdüz,..."
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,70.1,"bisikletle, ulaşımda, yakını","bu, nedenle, işine, okuluna, yarısına, gidiyor..."
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,83.4,danimarkayla,"alakalı, kadar, şimdilik, bu, söyleyeceklerim"
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,55.7,"beğenmeyi, kanalıma, abone, gittiyse","video, hoşunuza, ve, olmayı, unutmayın"


In [129]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,57.2,"lm, betimlemesi, staff","bu, tarafından, dizinin, fi"
1,1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU,25.1,"yaptırılmıştır, betimleme, derneğine",sesli
2,2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU,0.1,wwwsebederorg,NaN
3,3,10.267,11.394,mumu,fI1BI4d5KFU,0.1,mumu,NaN
4,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,71.5,"safhada, katsayım","hat, bak, şu, an, sinir"
...,...,...,...,...,...,...,...,...
3934202,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,62.6,"bitirmeden, engebeli, yolların, arazilerin, bu...","isterim, ki, videoyu, olduğu, da, bir, dümdüz,..."
3934203,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,70.1,"bisikletle, ulaşımda, yakını","bu, nedenle, işine, okuluna, yarısına, gidiyor..."
3934204,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,83.4,danimarkayla,"alakalı, kadar, şimdilik, bu, söyleyeceklerim"
3934205,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,55.7,"beğenmeyi, kanalıma, abone, gittiyse","video, hoşunuza, ve, olmayı, unutmayın"


In [130]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU,57.2,"lm, betimlemesi, staff","bu, tarafından, dizinin, fi"
1,4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU,71.5,"safhada, katsayım","hat, bak, şu, an, sinir"
2,5,14.213,17.219,kelimelerini lütfen seçerek konuş lütfen,fI1BI4d5KFU,40.1,"kelimelerini, seçerek","lütfen, konuş"
3,6,18.687,19.854,otursana bir tanem,fI1BI4d5KFU,100.1,NaN,"tanem, otursana, bir"
4,7,19.967,22.805,hah oturtmaya çalıştığına göre demek doğru,fI1BI4d5KFU,66.8,"oturtmaya, çalıştığına","hah, demek, göre, doğru"
...,...,...,...,...,...,...,...,...
3758042,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,62.6,"bitirmeden, engebeli, yolların, arazilerin, bu...","isterim, ki, videoyu, olduğu, da, bir, dümdüz,..."
3758043,3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0,70.1,"bisikletle, ulaşımda, yakını","bu, nedenle, işine, okuluna, yarısına, gidiyor..."
3758044,3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0,83.4,danimarkayla,"alakalı, kadar, şimdilik, bu, söyleyeceklerim"
3758045,3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0,55.7,"beğenmeyi, kanalıma, abone, gittiyse","video, hoşunuza, ve, olmayı, unutmayın"


In [131]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
5,8,23.921,25.737,doğru mu bu konuşsana ya,fI1BI4d5KFU,100.1,NaN,"mu, ya, bu, doğru, konuşsana",doğru mu bu konuşsana ya
21,24,75.854,78.412,bu gerçeği size nasıl söyleyeceğim diye,fI1BI4d5KFU,100.1,NaN,"nasıl, size, diye, bu, söyleyeceğim, gerçeği",bu gerçeği size nasıl söyleyeceğim diye
22,25,81.538,84.046,demek o yüzden öyle garip davrandın,fI1BI4d5KFU,100.1,NaN,"o, davrandın, demek, garip, öyle, yüzden",demek o yüzden öyle garip davrandın
24,27,87.460,91.423,evde değerli ne varsa bagaja tıkmalar,fI1BI4d5KFU,83.4,tıkmalar,"varsa, ne, evde, değerli, bagaja",evde değerli ne varsa bagaja
30,33,114.226,116.574,yirmi beş yıl dile kolay,fI1BI4d5KFU,100.1,NaN,"kolay, yirmi, dile, beş, yıl",yirmi beş yıl dile kolay
...,...,...,...,...,...,...,...,...,...
3758039,3934199,583.933,590.183,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0,86.8,"ülkemizde, yapmıyordur","varsa, eğer, çok, kıyafet, da, el, ikinci, hat...",ikinci el kıyafet dükkanı gördüğümü hatırlamıy...
3758040,3934200,590.325,594.700,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0,80.1,"insanlarımız, yaramayan","bizim, el, zira, kıyafetleri, ikinci, işine, y...",ikinci el kıyafetleri satmak yerine
3758041,3934201,594.755,599.701,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0,83.4,"sahiplerine, örneğidir","iyilik, çok, tercih, vermeyi, ihtiyaç, bir, ed...",vermeyi tercih ediyorlar ve bu çok güzel bir i...
3758042,3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0,62.6,"bitirmeden, engebeli, yolların, arazilerin, bu...","isterim, ki, videoyu, olduğu, da, bir, dümdüz,...",şunu da söylemek isterim ki


In [132]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,doğru mu bu konuşsana ya,23.921000,25.737000,doğru mu bu konuşsana ya,fI1BI4d5KFU
1,bu gerçeği size nasıl söyleyeceğim diye,75.854000,78.412000,bu gerçeği size nasıl söyleyeceğim diye,fI1BI4d5KFU
2,demek o yüzden öyle garip davrandın,81.538000,84.046000,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
3,evde değerli ne varsa bagaja,87.460000,90.566135,evde değerli ne varsa bagaja tıkmalar,fI1BI4d5KFU
4,yirmi beş yıl dile kolay,114.226000,116.574000,yirmi beş yıl dile kolay,fI1BI4d5KFU
...,...,...,...,...,...
1162908,ikinci el kıyafet dükkanı gördüğümü hatırlamıy...,584.825857,589.528238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
1162909,ikinci el kıyafetleri satmak yerine,592.600000,594.700000,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0
1162910,vermeyi tercih ediyorlar ve bu çok güzel bir i...,595.915173,599.151444,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0
1162911,şunu da söylemek isterim ki,601.376700,603.208050,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0


In [133]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,doğru mu bu konuşsana ya,23.621000,26.037000,doğru mu bu konuşsana ya,fI1BI4d5KFU
1,bu gerçeği size nasıl söyleyeceğim diye,75.554000,78.712000,bu gerçeği size nasıl söyleyeceğim diye,fI1BI4d5KFU
2,demek o yüzden öyle garip davrandın,81.238000,84.346000,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
3,evde değerli ne varsa bagaja,87.160000,90.866135,evde değerli ne varsa bagaja tıkmalar,fI1BI4d5KFU
4,yirmi beş yıl dile kolay,113.926000,116.874000,yirmi beş yıl dile kolay,fI1BI4d5KFU
...,...,...,...,...,...
1162908,ikinci el kıyafet dükkanı gördüğümü hatırlamıy...,584.525857,589.828238,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
1162909,ikinci el kıyafetleri satmak yerine,592.300000,595.000000,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0
1162910,vermeyi tercih ediyorlar ve bu çok güzel bir i...,595.615173,599.451444,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0
1162911,şunu da söylemek isterim ki,601.076700,603.508050,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0


In [134]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,doğru mu bu konuşsana ya,24,26,doğru mu bu konuşsana ya,fI1BI4d5KFU
1,bu gerçeği size nasıl söyleyeceğim diye,76,79,bu gerçeği size nasıl söyleyeceğim diye,fI1BI4d5KFU
2,demek o yüzden öyle garip davrandın,81,84,demek o yüzden öyle garip davrandın,fI1BI4d5KFU
3,evde değerli ne varsa bagaja,87,91,evde değerli ne varsa bagaja tıkmalar,fI1BI4d5KFU
4,yirmi beş yıl dile kolay,114,117,yirmi beş yıl dile kolay,fI1BI4d5KFU
...,...,...,...,...,...
1162908,ikinci el kıyafet dükkanı gördüğümü hatırlamıy...,585,590,fakat ülkemizde ikinci el kıyafet dükkanı görd...,dRssR_apbR0
1162909,ikinci el kıyafetleri satmak yerine,592,595,zira bizim insanlarımız işine yaramayan ikinci...,dRssR_apbR0
1162910,vermeyi tercih ediyorlar ve bu çok güzel bir i...,596,599,ihtiyaç sahiplerine vermeyi tercih ediyorlar v...,dRssR_apbR0
1162911,şunu da söylemek isterim ki,601,604,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0


In [135]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,beslemek istiyorum çünkü harika bir iş çıkarıy...,86,87,trump peter sadece izin ver sadece size şunu s...,oobHYBDiDAc
1,bu yapay paralar vesaire onlar da birazcık asl...,2523,2536,bir de tabi işin ekonomik tarafında da işte bl...,MEW9uH5mfBQ
2,olun herkes en süper kendisi onun için en süpe...,368,381,başkası diyor falan yani müsterih olun herkes ...,ek3iKA0vBE4
3,orayı zaten belli değil mi oraya göreceğiz çok...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc
4,üzere kalenin etrafında herhangi bir içeri gir...,372,372,merhaba arkadaşlar youtube kanalımıza hoş geld...,S6wCHPZLjsw
...,...,...,...,...,...
1162908,i ı ı ı ı,107,110,i ı ı ı ı,Jup8NeCvwHM
1162909,m n m o m,544,545,m n m o m cnun olup,ZjTPM8bhXp4
1162910,a a a a a,2305,2309,cahil herhalde vallahi biz kimiz konuştuk sonr...,ITI_BsGwXFQ
1162911,r o l e x,1,6,jeo r o l e x rolex,ApfKkodSYgc


In [136]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,beslemek istiyorum çünkü harika bir iş çıkarıy...,86,87,trump peter sadece izin ver sadece size şunu s...,oobHYBDiDAc,81
1,bu yapay paralar vesaire onlar da birazcık asl...,2523,2536,bir de tabi işin ekonomik tarafında da işte bl...,MEW9uH5mfBQ,86
2,olun herkes en süper kendisi onun için en süpe...,368,381,başkası diyor falan yani müsterih olun herkes ...,ek3iKA0vBE4,85
3,orayı zaten belli değil mi oraya göreceğiz çok...,987,988,yerleştirmeye yumurtasını seviyoruz çok güzel ...,eqkDognLtTc,75
4,üzere kalenin etrafında herhangi bir içeri gir...,372,372,merhaba arkadaşlar youtube kanalımıza hoş geld...,S6wCHPZLjsw,100
...,...,...,...,...,...,...
1162908,i ı ı ı ı,107,110,i ı ı ı ı,Jup8NeCvwHM,40
1162909,m n m o m,544,545,m n m o m cnun olup,ZjTPM8bhXp4,60
1162910,a a a a a,2305,2309,cahil herhalde vallahi biz kimiz konuştuk sonr...,ITI_BsGwXFQ,20
1162911,r o l e x,1,6,jeo r o l e x rolex,ApfKkodSYgc,100


In [137]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,bu yapay paralar vesaire onlar da birazcık asl...,2523,2536,bir de tabi işin ekonomik tarafında da işte bl...,MEW9uH5mfBQ
1,üzere kalenin etrafında herhangi bir içeri gir...,372,372,merhaba arkadaşlar youtube kanalımıza hoş geld...,S6wCHPZLjsw
2,ışığı çok sever ancak doğrudan ve şiddetli gün...,204,204,şans bambusu bakımı nasıl yapılır şans bambusu...,fcm9ZRHe_GY
3,neler hissediyorum şu anda tam olarak neler ya...,672,684,neler hissediyorum şu anda tam olarak neler ya...,CULrx3yerE8
4,dediğin gibi eğitim vermeye çalışıyoruz ama in...,2844,2855,şimdi biz eser sahiplerine dediğin gibi eğitim...,MEW9uH5mfBQ
...,...,...,...,...,...
993889,oo o ne ki i,2730,2732,oo o ne ki i lyas,FkKOGn0GCw8
993890,e o h l var,384,386,e o h l var,le0BtRt794k
993891,a c ve e k,421,422,a c ve e k vitamini bakımından zenginimantioks...,zzOL9mRRU0c
993892,a b c d e,66,69,a b c d e,hbo7XlB0FWA


In [138]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-138-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,bu yapay paralar vesaire onlar da birazcık asl...,2523,2536,bir de tabi işin ekonomik tarafında da işte bl...,MEW9uH5mfBQ,54
1,üzere kalenin etrafında herhangi bir içeri gir...,372,372,merhaba arkadaşlar youtube kanalımıza hoş geld...,S6wCHPZLjsw,21
2,ışığı çok sever ancak doğrudan ve şiddetli gün...,204,204,şans bambusu bakımı nasıl yapılır şans bambusu...,fcm9ZRHe_GY,10
3,neler hissediyorum şu anda tam olarak neler ya...,672,684,neler hissediyorum şu anda tam olarak neler ya...,CULrx3yerE8,80
4,dediğin gibi eğitim vermeye çalışıyoruz ama in...,2844,2855,şimdi biz eser sahiplerine dediğin gibi eğitim...,MEW9uH5mfBQ,28
...,...,...,...,...,...,...
993889,oo o ne ki i,2730,2732,oo o ne ki i lyas,FkKOGn0GCw8,70
993890,e o h l var,384,386,e o h l var,le0BtRt794k,100
993891,a c ve e k,421,422,a c ve e k vitamini bakımından zenginimantioks...,zzOL9mRRU0c,7
993892,a b c d e,66,69,a b c d e,hbo7XlB0FWA,100


In [139]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

/home/kurubal/anaconda3/envs/py39/lib/python3.9/site-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,search_string,start_time,end_time,sentence,video_id
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA
1,ancak herhangi bir sorusu varsa ya da belki bi...,682,691,ancak herhangi bir sorusu varsa ya da belki bi...,J0hq13epy2k
2,oradaki sadece şu çok önemliydi dört kere geld...,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...,LOQOS5IJtlM
3,sen hayattan ne istiyorsan onun için çalışmalı...,1342,1377,sen hayattan ne istiyorsan onun için çalışmalı...,luv56Zk_gkY
4,herhalde geldiğim en yoksul ülke burası oldu y...,1074,1115,herhalde geldiğim en yoksul ülke burası oldu y...,9jCA8ZQD2IY
...,...,...,...,...,...
472091,bu ne h l ya,2513,2515,bu ne h l ya,Bbaal2Neohk
472092,bu ne h l ha,9211,9214,bu ne h l ha,JrJs2vfea4g
472093,bu ne h l be,67,68,bu ne h l be,q0hmH24_RRg
472094,e o h l var,384,386,e o h l var,le0BtRt794k


In [140]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,genellikle teknolojik gelişmeleri tek yönlü ol...,zzjsW3gqurc
1,ana yaşam alanına geldik açık mutfak salon,zzXTIIo7EvM
2,sağ tarafına gidiyorum yukarıdaki ikinci odamı...,zzXTIIo7EvM
3,küçükken hayal ettiğim meslek fırtına,zyite5M_ThA
4,elbette sosyal medya aracılığıyla doğrudan ile...,zxznlmnXsoo
...,...,...
14585,san francisco uluslararası havaalanına götürec...,01qjQ_KPgpY
14586,gece yarısı sabaha karşı saat,01ghGqxKtJU
14587,yaptığım basın toplantısında türk hava,00Dl6tNNlZ8
14588,sizler doğru haberi evlerinde oturan sokaklard...,00Dl6tNNlZ8


In [141]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA
1,ancak herhangi bir sorusu varsa ya da belki bi...,682,691,ancak herhangi bir sorusu varsa ya da belki bi...,J0hq13epy2k
2,oradaki sadece şu çok önemliydi dört kere geld...,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...,LOQOS5IJtlM
3,sen hayattan ne istiyorsan onun için çalışmalı...,1342,1377,sen hayattan ne istiyorsan onun için çalışmalı...,luv56Zk_gkY
4,herhalde geldiğim en yoksul ülke burası oldu y...,1074,1115,herhalde geldiğim en yoksul ülke burası oldu y...,9jCA8ZQD2IY
...,...,...,...,...,...
14484,im in a new house,758,761,im in a new house,LFgUg6RjgA8
14485,how we gon na do,82,85,how we gon na do,3CmoMhIrf9U
14486,cennet ü d z h,3474,3476,cennet ü d z h,11aXftLrWIc
14487,p ç t k f h s,19,30,p ç t k f h s ş,f5QEHWF2JIs


In [142]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,on tane geliyor haftada üç dört kere şeye çıkı...,679,690,on tane geliyor haftada üç dört kere şeye çıkı...,LPrMSJHDDcA,https://www.youtube.com/watch?v=LPrMSJHDDcA&t=...
1,ancak herhangi bir sorusu varsa ya da belki bi...,682,691,ancak herhangi bir sorusu varsa ya da belki bi...,J0hq13epy2k,https://www.youtube.com/watch?v=J0hq13epy2k&t=...
2,oradaki sadece şu çok önemliydi dört kere geld...,3834,3843,oradaki sadece şu çok önemliydi dört kere geld...,LOQOS5IJtlM,https://www.youtube.com/watch?v=LOQOS5IJtlM&t=...
3,sen hayattan ne istiyorsan onun için çalışmalı...,1342,1377,sen hayattan ne istiyorsan onun için çalışmalı...,luv56Zk_gkY,https://www.youtube.com/watch?v=luv56Zk_gkY&t=...
4,herhalde geldiğim en yoksul ülke burası oldu y...,1074,1115,herhalde geldiğim en yoksul ülke burası oldu y...,9jCA8ZQD2IY,https://www.youtube.com/watch?v=9jCA8ZQD2IY&t=...
...,...,...,...,...,...,...
14484,im in a new house,758,761,im in a new house,LFgUg6RjgA8,https://www.youtube.com/watch?v=LFgUg6RjgA8&t=...
14485,how we gon na do,82,85,how we gon na do,3CmoMhIrf9U,https://www.youtube.com/watch?v=3CmoMhIrf9U&t=82s
14486,cennet ü d z h,3474,3476,cennet ü d z h,11aXftLrWIc,https://www.youtube.com/watch?v=11aXftLrWIc&t=...
14487,p ç t k f h s,19,30,p ç t k f h s ş,f5QEHWF2JIs,https://www.youtube.com/watch?v=f5QEHWF2JIs&t=19s


In [143]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [144]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [145]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [146]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_5000_Word6.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_5000_Word6.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_10000_Word6.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_10000_Word6.xlsx',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'Turkish_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx']

In [147]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [148]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Analysis

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list2 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list2.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list2), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result 

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
if word_group_adjust_select:
    df_adjust_text_ratio = df_exract_word_group(df_adjust_text_ratio, "sentence", "video_id", word_list, sent_len=word_group_adjust_select, sent_len_num=sentence_word_count_number)
else:
    df_adjust_text_ratio.loc[:,"search_string"] = df_adjust_text_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
    df_adjust_text_ratio = df_adjust_text_ratio[~(df_adjust_text_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
    
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
#df_word_group_time_loc_in_text_result.to_csv(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.csv", index=False)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

In [ ]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

#### Youtube Videoid Text Adjust Result Analysis Multiprocess

In [ ]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [ ]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

In [ ]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# sentence ratio and time shift
adjust_sent_word_ratio = 50
adjust_text_word_ratio = 5
shift = 0.3  # sentence time shift

# func select
word_group_adjust_select = True  # True, False; True for selecting word group according to sentence_word_count_number
sentence_word_count_number = 5   # False for selecting word group according to all word group length

# # shared word frequency
# shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"{file_ext}")

In [ ]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [ ]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

In [ ]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [ ]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

In [ ]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [ ]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [ ]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

In [ ]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

In [ ]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [ ]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

In [ ]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence.reset_index(inplace=True)
df_videoid_sentence

In [ ]:
df_videoid_start_time = df_youtube_sentence.groupby("video_id")[["start_time"]].min()
df_videoid_start_time.reset_index(inplace=True)
df_videoid_start_time

In [ ]:
df_videoid_end_time = df_youtube_sentence.groupby("video_id")[["end_time"]].max()
df_videoid_end_time.reset_index(inplace=True)
df_videoid_end_time

In [ ]:
df_merge_start_end_time = pd.merge(df_videoid_start_time, df_videoid_end_time, how="inner", on="video_id")
df_merge_start_end_time.drop_duplicates(inplace=True)
df_merge_start_end_time

In [ ]:
df_videoid_sentence = pd.merge(df_videoid_sentence, df_merge_start_end_time,how="inner", on="video_id")
df_videoid_sentence.drop_duplicates(inplace=True)
df_videoid_sentence

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list3 = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_videoid_sentence)))

def videoid_word_ratio(index_num):
    index = df_videoid_sentence.loc[index_num,"index"]
    videoid = df_videoid_sentence.loc[index_num,"video_id"]
    sentence = df_videoid_sentence.loc[index_num,"sentence"]
    

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/len(sent_word)+0.001)*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list3.append([index,videoid,sentence,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(videoid_word_ratio, index_num)

In [ ]:
df_text_ratio_result = pd.DataFrame(list(result_list3), columns=["index","video_id","sentence","word_ratio","different_word","intersect_word"])
df_text_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_text_ratio_result.reset_index(drop=True, inplace=True)
df_text_ratio_result.drop(["index"], axis=1, inplace=True)
df_text_ratio_result

In [ ]:
df_text_ratio_result["different_word"] = df_text_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result["intersect_word"] = df_text_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_text_ratio_result

In [ ]:
df_adjust_text_ratio = df_text_ratio_result[df_text_ratio_result["word_ratio"] >= adjust_text_word_ratio]
df_adjust_text_ratio.reset_index(inplace=True, drop=True)
df_adjust_text_ratio

In [ ]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list4 = manager.list()
index_list = manager.list()
all_index_list = manager.list()
var_index_list = manager.list()
text_all_list = manager.list()  
df = df_adjust_text_ratio
df.reset_index(drop=True, inplace=True)
index_num = list(range(len(df)))
sent_len = word_group_adjust_select
sent_len_num = sentence_word_count_number 

def df_exract_word_group(index_num):    
    source_text = df.loc[index_num,"sentence"]
    opt_var = df.loc[index_num,"video_id"]
    words = re.findall(r"\w+", source_text, re.UNICODE)
    
    index_list = []
    for j in range(len(words)):
        if words[j] in word_list:
            index_list.append(j)
        else:
            pass
    all_index_list = []
    var_index_list = []
    for k in range(len(index_list)):
        try:
            var1 = index_list[k] + 1  
            var2 = index_list[k+1]
        except:
            var1 = index_list[k] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[k]
            var_index_list.append(var3)
        else:
            var3 = index_list[k]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []
    text_all_list = []
    for m in all_index_list:
        text_list = [] 
        for n in m:
            word = words[n]
            text_list.append(word)
            if sent_len:
                if len(text_list) >= sent_len_num:
                    text = " ".join(text_list)
                    text_all_list.append(text)
                else:
                    pass
            else:
               text = " ".join(text_list)
               text_all_list.append(text) 
        
    for search_string in text_all_list:
        #df_var["index"] = i
        #df_var["search_string"] = search_string
        #df_var["sentence"] = source_text
        #df_var["video_id"] = opt_var
        result_list4.append([index_num,search_string,source_text,opt_var])

if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(df_exract_word_group, index_num)

In [ ]:
df_adjust_text_ratio = pd.DataFrame(list(result_list4), columns=["index","search_string","sentence","video_id"])
df_adjust_text_ratio.sort_values(by="index", ascending=True, inplace=True)
df_adjust_text_ratio.reset_index(drop=True, inplace=True)
df_adjust_text_ratio.drop(["index"], axis=1, inplace=True)
df_adjust_text_ratio

In [ ]:
df_adjust_text_ratio = df_adjust_text_ratio[["search_string","video_id","sentence"]]
df_adjust_text_ratio

In [ ]:
df_videoid_sentence = df_videoid_sentence[["video_id","start_time","end_time"]]

In [ ]:
df_adjust_text_ratio = pd.merge(df_adjust_text_ratio, df_videoid_sentence, how="inner", on="video_id")
df_adjust_text_ratio

In [ ]:
df_word_group_time_loc_in_text_result = word_group_time_loc(df_adjust_text_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["start_time"] = df_word_group_time_loc_in_text_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result["end_time"] = df_word_group_time_loc_in_text_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_text_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_in_text_result['video_id'].map(str)+"&t="+df_word_group_time_loc_in_text_result['start_time'].map(str)+"s"
df_word_group_time_loc_in_text_result

In [ ]:
df_word_group_time_loc_in_text_result = df_word_group_time_loc_in_text_result.head(1000000)

In [ ]:
if word_group_adjust_select:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Adjust_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)
else:
    df_word_group_time_loc_in_text_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_All_In_Youtube_Sentence_Text_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [ ]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.xlsx")
output_file2

In [ ]:
for l in output_file2:
    source = l # source directory
    destination = path
    shutil.copy2(source, destination)

In [ ]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

#### Temp

In [ ]:
df_28 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_28_Word6.csv")
df_28

In [ ]:
df_200 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_200_Word6.csv")
df_200

In [ ]:
df_1000 = pd.read_csv("Turkish_Word_Group_Max_In_Youtube_Sentence_1000_Word6.csv")
df_1000